# Module 4: 스택 기반 가상 머신 (Stack-Based VM) 시뮬레이션

이 노트북에서는 Python의 스택 기반 가상 머신(VM) 동작 원리를 시뮬레이션하고 시각화합니다.

## 학습 목표
- 스택 기반 VM의 기본 개념 이해
- 평가 스택(Evaluation Stack)과 프레임 스택(Frame Stack)의 차이점 파악
- 함수 호출, 재귀, 예외 처리 시 스택의 동작 관찰

---
## 1. 소개: 스택 기반 가상 머신

### 왜 Python은 스택 VM인가?

Python 인터프리터(CPython)는 **스택 기반 가상 머신(Stack-Based VM)**을 사용합니다.

**스택 VM의 장점:**
- **단순성**: 명령어가 피연산자를 명시할 필요 없음 (스택에서 자동으로 POP)
- **이식성**: 하드웨어에 독립적인 바이트코드 생성
- **컴팩트한 바이트코드**: 레지스터 번호 대신 스택 위치 사용

### 레지스터 VM vs 스택 VM

| 특성 | 레지스터 VM (예: Lua, JVM) | 스택 VM (예: Python, Ruby) |
|------|---------------------------|---------------------------|
| 명령어 크기 | 길음 (레지스터 번호 명시) | 짧음 |
| 피연산자 접근 | 직접 (레지스터 번호) | 간접 (스택 TOP) |
| 구현 복잡도 | 복잡 | 단순 |
| 코드 밀도 | 낮음 | 높음 |

### 예시: `a + b` 연산

**레지스터 VM:**
```
ADD R1, R2, R3   # R1 = R2 + R3
```

**스택 VM (Python):**
```
LOAD_FAST a      # 스택에 a PUSH
LOAD_FAST b      # 스택에 b PUSH
BINARY_ADD       # 두 값 POP, 더한 결과 PUSH
```

---
## 2. 평가 스택 (Evaluation Stack)

Python VM은 각 함수 실행 시 **평가 스택**을 사용하여 중간 결과를 저장합니다.

### 스택 연산 기본

| 연산 | 설명 | 스택 변화 |
|------|------|----------|
| `PUSH x` | 값 x를 스택에 push | `[] → [x]` |
| `POP` | 스택 top 값을 pop | `[a, b] → [a]` |
| `DUP` | 스택 top을 복제 | `[a] → [a, a]` |
| `SWAP` | 스택 top 2개 교환 | `[a, b] → [b, a]` |

### Python 바이트코드의 스택 효과

```python
# 예: a + b
LOAD_FAST a      # 스택: [] → [a]        (push 1)
LOAD_FAST b      # 스택: [a] → [a, b]    (push 1)
BINARY_ADD       # 스택: [a, b] → [a+b]  (pop 2, push 1)
```

**모든 바이트코드 명령어는 스택 효과(Stack Effect)를 가집니다.**
- 얼마나 많은 값을 pop하는가?
- 얼마나 많은 값을 push하는가?

In [ ]:
# 스택 효과 확인을 위한 헬퍼 함수
import dis

def show_stack_effect(func):
    """함수의 각 바이트코드 명령어와 스택 효과를 출력"""
    print(f"\n=== {func.__name__}의 바이트코드 ==="
    for instr in dis.get_instructions(func):
        print(f"{instr.offset:3d}: {instr.opname:20s}", end="")
        if instr.argval is not None:
            print(f" ({instr.argval})", end="")
        print()

# 간단한 예제
def simple_add(a, b):
    return a + b

show_stack_effect(simple_add)

---
## 3. 프레임 스택 (Frame Stack)

Python은 **프레임(Frame)**이라는 구조체로 함수 실행 컨텍스트를 관리합니다.

### 프레임의 구성 요소

```
┌─────────────────────────────────────┐
│           Frame Object              │
├─────────────────────────────────────┤
│ f_code         : 코드 객체          │
│ f_locals       : 지역 변수 dict     │
│ f_globals      : 전역 변수 dict     │
│ f_builtins     : 내장 함수 dict     │
│ f_back         : 이전 프레임 참조   │
│ f_lasti        : 마지막 실행 인덱스 │
│ f_stack        : 평가 스택 (C 스택) │
└─────────────────────────────────────┘
```

### 함수 호출 시 프레임 생성 과정

1. **새 프레임 할당**: 호출된 함수의 코드 객체 기반
2. **인자 바인딩**: `f_locals`에 인자값 저장
3. **프레임 푸시**: `f_back`에 이전 프레임 연결
4. **코드 실행**: 새 프레임의 평가 스택 사용
5. **프레임 팝**: 실행 완료 후 이전 프레임으로 복귀

### 지역변수와 스택의 관계

```python
def example(x):
    y = x + 1      # 1. LOAD_FAST x (스택에 x push)
                   # 2. LOAD_CONST 1 (스택에 1 push)
                   # 3. BINARY_ADD (pop 2, push 결과)
                   # 4. STORE_FAST y (pop, locals['y']에 저장)
    return y       # 5. LOAD_FAST y (스택에 y push)
                   # 6. RETURN_VALUE (pop, 반환)
```

In [ ]:
# 프레임 스택 확인하기
import sys

def show_frame_info():
    """현재 프레임 정보 출력"""
    frame = sys._getframe()
    print(f"현재 함수: {frame.f_code.co_name}")
    print(f"지역 변수: {list(frame.f_locals.keys())}")
    print(f"이전 프레임: {frame.f_back.f_code.co_name if frame.f_back else None}")

def outer():
    x = 10
    inner()

def inner():
    y = 20
    show_frame_info()

outer()

---
## 4. 실습 1: stack_animator 사용하기

`tools/stack_animator.py` 모듈을 사용하여 바이트코드 실행을 시각화합니다.

### animate_execution 함수

```python
from tools.stack_animator import animate_execution

animate_execution(func, args=(), delay=0.5)
```

- `func`: 시각화할 Python 함수
- `args`: 함수에 전달할 인자 튜플
- `delay`: 스텝 간 지연 시간(초). 0이면 즉시 출력

In [ ]:
import sys
sys.path.insert(0, '..')

from tools.stack_animator import animate_execution

# 간단한 덧셈 함수
def add(a, b):
    return a + b

print("=== 간단한 덧셈 함수의 스택 변화 ===")
animate_execution(add, args=(3, 5), delay=0)

### 실행 결과 해석

위 출력에서 각 스텝을 보면:
1. `LOAD_FAST a` - 지역변수 `a`의 값을 스택에 push
2. `LOAD_FAST b` - 지역변수 `b`의 값을 스택에 push
3. `BINARY_OP` - 스택에서 2개 pop, 더한 결과 push
4. `RETURN_VALUE` - 스택 top 값을 반환

In [ ]:
# 지역 변수가 있는 함수
def compute(x, y):
    sum_val = x + y
    result = sum_val * 2
    return result

print("=== 지역 변수를 사용하는 함수 ===")
animate_execution(compute, args=(3, 4), delay=0)

---
## 5. 실습 2: 복잡한 표현식의 스택 변화

수식 `(a + b) * (c - d)`의 스택 변화를 추적합니다.

### 중위 표기법 vs 후위 표기법

스택 VM은 **후위 표기법(Postfix/Reverse Polish Notation)**을 자연스럽게 처리합니다.

| 표기법 | 형태 | 스택 연산 |
|--------|------|----------|
| 중위(Infix) | `(a + b) * (c - d)` | 괄호 필요 |
| 후위(Postfix) | `a b + c d - *` | 괄호 불필요 |

Python 컴파일러는 중위 표기법을 후위 형태의 바이트코드로 변환합니다.

In [ ]:
# 복잡한 표현식
def complex_expr(a, b, c, d):
    return (a + b) * (c - d)

print("=== 복잡한 표현식 (a + b) * (c - d) ===")
animate_execution(complex_expr, args=(2, 3, 10, 4), delay=0)

### 스택 변화 분석

`(2 + 3) * (10 - 4)` 실행 시 스택 상태:

```
Step 1: LOAD_FAST a      → 스택: [2]
Step 2: LOAD_FAST b      → 스택: [2, 3]
Step 3: BINARY_OP(+)     → 스택: [5]        (2+3)
Step 4: LOAD_FAST c      → 스택: [5, 10]
Step 5: LOAD_FAST d      → 스택: [5, 10, 4]
Step 6: BINARY_OP(-)     → 스택: [5, 6]      (10-4)
Step 7: BINARY_OP(*)     → 스택: [30]        (5*6)
Step 8: RETURN_VALUE     → 반환: 30
```

In [ ]:
# 더 복잡한 표현식: 중첩 연산
def nested_ops(x, y, z):
    return x * y + z * (x - y)

print("=== 중첩 연산 x*y + z*(x-y) ===")
animate_execution(nested_ops, args=(2, 3, 4), delay=0)

---
## 6. 실습 3: 재귀 함수의 프레임 스택

재귀 함수는 **프레임 스택**을 시각적으로 보여주는 좋은 예제입니다.

### 재귀와 프레임 스택

```
factorial(3) 호출 시 프레임 스택:

  ┌─────────────┐  ← TOP (현재 실행)
  │ factorial(1)│  n=1, 반환값: 1
  ├─────────────┤
  │ factorial(2)│  n=2, 기다리는 중: 2 * factorial(1)
  ├─────────────┤
  │ factorial(3)│  n=3, 기다리는 중: 3 * factorial(2)
  ├─────────────┤
  │   <main>    │  최초 호출자
  └─────────────┘
```

⚠️ **주의**: 너무 깊은 재귀는 `RecursionError`를 발생시킵니다.
Python의 기본 재귀 한도는 1000입니다.

In [ ]:
# 재귀 함수 - 팩토리얼
def factorial(n):
    if n <= 1:
        return 1
    return n * factorial(n - 1)

print("=== 재귀 함수 factorial(3) ===")
print("참고: 재귀 호출은 각각 독립적인 프레임을 생성합니다")
animate_execution(factorial, args=(3,), delay=0)

### 재귀 깊이 확인

각 재귀 호출마다 새로운 프레임이 생성되고,
반환 시 프레임이 제거됩니다.

In [ ]:
import sys

# 재귀 깊이 추적
def factorial_with_depth(n, depth=0):
    indent = "  " * depth
    frame = sys._getframe()
    print(f"{indent}프레임 {depth}: factorial({n}) - f_back: {frame.f_back.f_code.co_name if frame.f_back else None}")
    
    if n <= 1:
        result = 1
    else:
        result = n * factorial_with_depth(n - 1, depth + 1)
    
    print(f"{indent}프레임 {depth} 반환: {result}")
    return result

print("=== 재귀 호출의 프레임 스택 변화 ===")
factorial_with_depth(3)

In [ ]:
# 피보나치 수열 (재귀)
def fibonacci(n):
    if n <= 1:
        return n
    return fibonacci(n - 1) + fibonacci(n - 2)

print("=== 피보나치 fibonacci(4) ===")
print("참고: fibonacci는 분기 재귀로 프레임 트리를 생성합니다")
animate_execution(fibonacci, args=(4,), delay=0)

---
## 7. 실습 4: 예외와 스택 언와인딩

예외가 발생하면 Python은 **스택 언와인딩(Stack Unwinding)**을 수행합니다.

### 스택 언와인딩 과정

1. **예외 발생**: 현재 프레임에서 예외 객체 생성
2. **프레임 탐색**: 예외 핸들러(`try-except`) 검색
3. **프레임 팝**: 핸들러를 찾을 때까지 프레임 스택 pop
4. **핸들러 실행**: 적절한 `except` 블록 실행
5. **정상 흐름**: 핸들러 이후 코드 계속 실행

```
호출 스택:          예외 발생 후:
┌─────────┐        ┌─────────┐
│  func3  │  ← 예외 │  func3  │  ← 예외 전파 중
├─────────┤        ├─────────┤
│  func2  │        │  func2  │  ← 핸들러 없음, pop
├─────────┤        ├─────────┤
│  func1  │        │  func1  │  ← 핸들러 발견!
├─────────┤        ├─────────┤
│  main   │        │  main   │
└─────────┘        └─────────┘
```

In [ ]:
# 예외 발생과 스택 언와인딩 시뮬레이션
import sys
import traceback

def level3():
    print("  level3: 예외 발생 직전")
    raise ValueError("테스트 예외")

def level2():
    print(" level2: level3 호출")
    level3()
    print(" level2: 이 코드는 실행되지 않음")

def level1():
    print("level1: level2 호출 (try 블록 안)")
    try:
        level2()
    except ValueError as e:
        print(f"level1: 예외 포착! - {e}")
    print("level1: 정상 계속")

print("=== 예외 처리와 스택 언와인딩 ===")
level1()

In [ ]:
# 예외 발생 시 스택 트레이스 분석
def func_c():
    raise RuntimeError("스택 트레이스 테스트")

def func_b():
    func_c()

def func_a():
    func_b()

print("=== 스택 트레이스 분석 ===")
try:
    func_a()
except RuntimeError:
    # 스택 트레이스 출력
    traceback.print_exc()
    
    # 프레임 체인 분석
    print("\n=== 프레임 체인 분석 ===")
    frame = sys._getframe()
    depth = 0
    while frame:
        print(f"깊이 {depth}: {frame.f_code.co_name} at {frame.f_code.co_filename}:{frame.f_lineno}")
        frame = frame.f_back
        depth += 1
        if depth > 10:  # 안전 제한
            break

### try-except의 바이트코드

`try-except`는 SETUP_FINALLY, POP_EXCEPT 등의 특수 바이트코드를 사용합니다.

In [ ]:
# try-except 바이트코드 확인
def try_except_example(x):
    try:
        result = 10 / x
    except ZeroDivisionError:
        result = float('inf')
    return result

print("=== try-except 바이트코드 ===")
dis.dis(try_except_example)

---
## 8. 연습 문제

다음 문제들을 직접 해결해보며 스택 VM의 동작을 이해해보세요.

### 문제 1: 스택 변화 예측

다음 함수 실행 시 각 스텝의 스택 상태를 예측해보세요:

```python
def mystery(a, b, c):
    x = a + b
    y = x * c
    return y - a
```

**힌트**: `dis.dis(mystery)`로 바이트코드를 확인한 후 스택 변화를 추적하세요.

In [ ]:
# 문제 1 풀이 공간
def mystery(a, b, c):
    x = a + b
    y = x * c
    return y - a

# 바이트코드 확인
print("=== mystery 함수의 바이트코드 ===")
dis.dis(mystery)

# 스택 애니메이션으로 확인
print("\n=== 스택 변화 시각화 ===")
animate_execution(mystery, args=(2, 3, 4), delay=0)

### 문제 2: 재귀 깊이 제한

Python의 재귀 한도를 확인하고, 안전한 재귀 함수를 작성하세요.

In [ ]:
# 문제 2 풀이 공간

# 1. 현재 재귀 한도 확인
print(f"현재 재귀 한도: {sys.getrecursionlimit()}")

# 2. 안전한 재귀 함수 작성 (깊이 제한)
def safe_factorial(n, max_depth=100):
    """안전한 팩토리얼 - 깊이 제한 포함"""
    def factorial_helper(x, current_depth):
        if current_depth > max_depth:
            raise RecursionError(f"최대 깊이 {max_depth} 초과")
        if x <= 1:
            return 1
        return x * factorial_helper(x - 1, current_depth + 1)
    
    return factorial_helper(n, 0)

# 테스트
print(f"safe_factorial(5) = {safe_factorial(5)}")
print(f"safe_factorial(10) = {safe_factorial(10)}")

### 문제 3: 꼬리 재귀 최적화 시뮬레이션

꼬리 재귀(Tail Recursion)를 반복문으로 변환하여 스택 오버플로우를 방지하세요.

In [ ]:
# 문제 3 풀이 공간

# 일반 재귀 팩토리얼
def factorial_recursive(n):
    if n <= 1:
        return 1
    return n * factorial_recursive(n - 1)

# 꼬리 재귀 팩토리얼
def factorial_tail(n, accumulator=1):
    if n <= 1:
        return accumulator
    return factorial_tail(n - 1, n * accumulator)

# 반복문으로 변환 (꼬리 재귀 최적화)
def factorial_iterative(n):
    accumulator = 1
    while n > 1:
        accumulator *= n
        n -= 1
    return accumulator

# 비교
print("=== 세 가지 팩토리얼 구현 비교 ===")
n = 5
print(f"factorial_recursive({n}) = {factorial_recursive(n)}")
print(f"factorial_tail({n}) = {factorial_tail(n)}")
print(f"factorial_iterative({n}) = {factorial_iterative(n)}")

# 스택 사용량 비교 (깊은 재귀)
print("\n=== 깊은 재귀 테스트 (n=100) ===")
try:
    # 재귀 한도 일시적으로 증가
    old_limit = sys.getrecursionlimit()
    sys.setrecursionlimit(200)
    
    # factorial_tail은 꼬리 재귀이지만 Python은 최적화하지 않음
    # 따라서 동일한 스택 사용량
    print(f"factorial_tail(100) = {factorial_tail(100)}")
    
    sys.setrecursionlimit(old_limit)
except RecursionError as e:
    print(f"RecursionError: {e}")
    sys.setrecursionlimit(old_limit)

# 반복문은 스택 문제 없음
print(f"factorial_iterative(1000) = {factorial_iterative(1000)}")

---
## 9. 요약

이 노트북에서 배운 내용:

### 핵심 개념

1. **스택 기반 VM**
   - 모든 연산은 스택을 통해 수행
   - 바이트코드는 스택 효과(Stack Effect)를 가짐
   - 레지스터 VM보다 구현이 단순

2. **평가 스택**
   - 함수 실행 중 중간 결과 저장
   - PUSH/POP 연산으로 데이터 흐름 제어
   - 후위 표기법 자연스럽게 처리

3. **프레임 스택**
   - 함수 호출 시 새 프레임 생성
   - 지역 변수, 코드 객체, 이전 프레임 참조 포함
   - 재귀 호출 시 프레임 체인 형성

4. **스택 언와인딩**
   - 예외 발생 시 프레임 스택 역순 탐색
   - 적절한 핸들러 찾을 때까지 프레임 pop
   - 예외 처리 후 정상 흐름 복귀

### 다음 단계

- Module 5: 바이트코드 최적화 기법
- Module 6: 직접 VM 구현하기

---
## 참고 자료

- [CPython Internals - Frame Objects](https://docs.python.org/3/c-api/frame.html)
- [Python Disassembler Documentation](https://docs.python.org/3/library/dis.html)
- [Inside the Python VM](https://leanpub.com/insidethepythonvm/read)